In [1]:
# Essentials
import numpy as np
from matplotlib import pyplot
import tensorflow
from tensorflow import keras
from keras import regularizers
from keras.models import Sequential
from keras.utils import np_utils
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionV3, ResNet50, InceptionResNetV2, Xception
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation
from keras.callbacks import  Callback, EarlyStopping
import matplotlib.pyplot as plt
from matplotlib import gridspec
import random
import imageio
import os
import cv2
import glob
random.seed(42)

In [2]:
%pip install wandb -q
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 2.1MB 2.9MB/s 
     |████████████████████████████████| 163kB 18.6MB/s 
     |████████████████████████████████| 133kB 16.2MB/s 
     |████████████████████████████████| 102kB 7.4MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 


In [3]:
# Fetch the dataset form Github
!git clone https://github.com/borate267/inaturalist-dataset.git

Cloning into 'inaturalist-dataset'...
remote: Enumerating objects: 12027, done.
remote: Total 12027 (delta 0), reused 0 (delta 0), pack-reused 12027
Receiving objects: 100% (12027/12027), 3.55 GiB | 26.81 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (11999/11999), done.


In [4]:
 # Load training and testing dataset

def data_for_testing(data_aug, batch_size):
  if data_aug:
      datagen = ImageDataGenerator(
                rotation_range=45, 
                width_shift_range=0.2, 
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2, 
                horizontal_flip=True, 
                fill_mode='reflect',
                rescale=1./255
                )
  else:
      datagen = ImageDataGenerator(rescale=1./255)

  train_data = datagen.flow_from_directory(
              directory='inaturalist-dataset/train',
              target_size = (128,128),
              batch_size=batch_size,
              class_mode="categorical",
              shuffle=True,
              seed=123)


  test_data = datagen.flow_from_directory(
            directory='inaturalist-dataset/val',
            target_size=(128,128),
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=1234)
  return test_data, train_data

In [7]:
def test():
    freeze_before = 100
    dropout = 0.2
    batch_size = 64
    num_dense = 512
    num_classes = 10
    input_shape = (128,128,3)
    base_model =  InceptionResNetV2(input_shape = input_shape, include_top = False, weights = 'imagenet')

    base_model.trainable = True

    # Freeze layers
    freeze_point = len(base_model.layers) - freeze_before
    for layer in base_model.layers[:freeze_point]:
        layer.trainable = False

    stack_model = Sequential()
    stack_model.add(base_model)
    stack_model.add(Flatten())
    stack_model.add(Dense(num_dense, activation='relu'))
    stack_model.add(Dense(num_classes, activation='softmax'))

    final_model = stack_model
    final_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

    train_data, test_data = data_for_testing('false', batch_size)

    final_model.fit(train_data, epochs=100)
    
    final_model.evaluate(test_data, batch_size = 64)

    final_model.save("model-best.h5")




In [ ]:
test()